### Exerciese 2 - Optimizing Gate Fidelity
Given that we now have a working measure of the closeness between a target gate and some implementation, we can now optimize the fidelity to find the parameters of an operation so that is maximizes the fidelity ( or minimize the error ). 

We will use scipy.optimize.minimize to do the optimization.

It is known that any single-qubit gate can be decomposed into three rotation angles on two axes via Euler angles. We will use a sequence of Z X Z rotations here, which can be implemented via QuTip as something like rz()*rx()*rz()

#### Step 0 : import packages

In [1]:
from qutip import *

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

import itertools

### Single-Qubit Case
#### Step 1 : Write a funciton which calculate the average gate fidelity, using oour states method, between the Z X Z rotation gate and a random unitary for the three angles.
The ranges for the Euler angles in SU(2) should be $(\alpha, \beta, \gamma)$ are $[0,\pi),[0,\pi/2],[0,\pi)$

In [28]:
rand_ang_list = np.random.rand(1,3)
Euler_angle_list = rand_ang_list[0].tolist()
Euler_angle_list = [angle * np.pi for angle in Euler_angle_list]
Euler_angle_list[1] /= 2
print(Euler_angle_list)

# create the Z X Z rotation operator
z_rot1 = gates.rz(Euler_angle_list[0])
x_rot = gates.rx(Euler_angle_list[1])
z_rot2 = gates.rz(Euler_angle_list[2])
zxz_rot_gate = z_rot2 * x_rot * z_rot1
print(zxz_rot_gate)
print(zxz_rot_gate * zxz_rot_gate.dag())  # should be identity

[0.273195457016863, 0.35192215415515593, 2.507200854840185]
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=False
Qobj data =
[[ 0.17684452-0.96854631j -0.15733729-0.07674008j]
 [ 0.15733729-0.07674008j  0.17684452+0.96854631j]]
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 1.00000000e+00 -3.10495214e-18]
 [-3.10495214e-18  1.00000000e+00]]


#### function for calculating gate fidelity

In [17]:
def make_input_dm_list(num_qubits: int, dim: int):
    """
    d+1 input density matrices used in your notebook:
      - d computational basis states |i><i|
      - 1 equal superposition state |d><d| with |d> = (1/sqrt(d)) sum_i |i>
    Returns: list[Qobj] of density matrices
    """
    input_state_list = []
    input_dm_list = []

    ket_tr = Qobj(arg=np.zeros((2**num_qubits, 1)), dims=[[2]*num_qubits, [1]])

    for bits in itertools.product([0, 1], repeat=num_qubits):
      
      # append B,i
      cur_ket = basis(2, bits[0])
      for qIdx in range(1, num_qubits):
         cur_ket = tensor(cur_ket, basis(2, bits[qIdx]))

      input_state_list.append(cur_ket)
      input_dm_list.append(cur_ket * cur_ket.dag())

      ket_tr = ket_tr + cur_ket

    # append the equal superposition state
    ket_tr = (1/np.sqrt(2**num_qubits)) * ket_tr
    
    dm_tr = 1 / dim * Qobj(arg=np.ones((2**num_qubits,2**num_qubits)), dims=[[2]*num_qubits, [2]*num_qubits])
    input_dm_list.append(dm_tr)

    return input_state_list, input_dm_list

In [18]:
# num_qubits = 3
# for bits in itertools.product([0, 1], repeat=num_qubits):
#       # append B,i
#       print(bits)

# Qobj(arg=np.zeros((2**num_qubits, 1)), dims=[[2]*num_qubits, [1]])

In [ ]:
num_qubits = 1
dim = 2 ** num_qubits
input_state_list, input_dm_list = make_input_dm_list(num_qubits, dim)
# print(input_state_list)
# print(input_dm_list)

[Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[1. 0.]
 [0. 0.]], Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0. 0.]
 [0. 1.]], Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.5 0.5]
 [0.5 0.5]]]


the single qubit gate

In [ ]:
gate1 = sigmax()

rand_ang_list = np.random.rand(1,3)
Euler_angle_list = rand_ang_list[0].tolist()
Euler_angle_list = [angle * np.pi for angle in Euler_angle_list]
Euler_angle_list[1] /= 2
print(Euler_angle_list)

# create the Z X Z rotation operator
z_rot1 = gates.rz(Euler_angle_list[0])
x_rot = gates.rx(Euler_angle_list[1])
z_rot2 = gates.rz(Euler_angle_list[2])
gate2 = z_rot2 * x_rot * z_rot1
# gate2 = gate1
print(gate2)
# print(gate2 * gate2.dag())  # should be identity

def get_zxz_gate(alpha: float, beta: float, gamma: float):
    # Rz(alpha) Rx(beta) Rz(gamma)
    return gates.rz(alpha) * gates.rx(beta) * gates.rz(gamma)

print(get_zxz_gate(Euler_angle_list[0], Euler_angle_list[1], Euler_angle_list[2]))

[0.23937480137908887, 1.4384727855701858, 2.8333502424170494]
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=False
Qobj data =
[[ 0.02589974-0.75186306j -0.63426841-0.17814241j]
 [ 0.63426841-0.17814241j  0.02589974+0.75186306j]]
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=False
Qobj data =
[[ 0.02589974-0.75186306j  0.63426841-0.17814241j]
 [-0.63426841-0.17814241j  0.02589974+0.75186306j]]


When $\beta = \pi$, for $\alpha = \gamma = $ any value, the rz() * rx() * rz() always gives the same result

In [ ]:
get_zxz_gate(-0.3, np.pi, -0.3)

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=False
Qobj data =
[[5.84974887e-17+1.80953938e-17j 0.00000000e+00-1.00000000e+00j]
 [0.00000000e+00-1.00000000e+00j 5.84974887e-17-1.80953938e-17j]]

In [48]:
def compute_state_fidelity_list(U: Qobj, V: Qobj, dim: int, input_dm_list=None):
    """
    Compute the list of state fidelities used in the notebook:
        F_i = F( U rho_i U† , V rho_i V† )
    where rho_i are the d computational basis states plus one equal-superposition state.

    Parameters
    ----------
    U, V : Qobj
        Two unitary gates to be compared.
    input_dm_list : list[Qobj]
        list of input density matrices.
    
    Returns
    -------
    state_fidelity_list : list[float]
        [F_0, F_1, ..., F_{d-1}, F_d]
    """

    # --------------------------------------------------
    # compute fidelities
    # --------------------------------------------------
    state_fidelity_list = []

    for rho in input_dm_list:
        rho_U = U * rho * U.dag()
        rho_V = V * rho * V.dag()
        F = fidelity(rho_U, rho_V)
        state_fidelity_list.append(float(F))

    return state_fidelity_list

def compute_gate_fidelity_arith_from_state_fidelity_list(state_fidelity_list: list, dim: int):
    # calculate the average gate fidelity with arithmetic mean
    fidelity_arith = 1 / ( dim + 1 ) * ( sum(state_fidelity_list) )
    return fidelity_arith

def compute_gate_fidelity_geom_from_state_fidelity_list(state_fidelity_list: list, dim: int):
    # calculate the average gate fidelity with modified geometric mean
    fidelity_prod = np.prod(state_fidelity_list[:-1])
    fidelity_geom = 1 / ( dim + 1 ) + (1 - 1 / ( dim + 1 )) * state_fidelity_list[-1] * fidelity_prod
    return fidelity_geom

def compute_gate_fidelity_lambda_from_state_fidelity_list(state_fidelity_list: list, dim: int):
    # calculate the average gate fidelity with a combined measure of two
    fidelity_prod = np.prod(state_fidelity_list[:-1])
    fidelity_geom = 1 / ( dim + 1 ) + (1 - 1 / ( dim + 1 )) * state_fidelity_list[-1] * fidelity_prod
    fidelity_arith = 1 / ( dim + 1 ) * ( sum(state_fidelity_list) )

    lambda_mix = 1 - (1 - fidelity_prod) / (1 - fidelity_prod * state_fidelity_list[-1])
    # lambda_mix = fidelity_prod * (1 - state_fidelity_list[-1]) / (1 - fidelity_prod * state_fidelity_list[-1])
    fidelity_lambda = lambda_mix * fidelity_geom + (1 - lambda_mix) * fidelity_arith

    return fidelity_lambda

def compute_gate_fidelity_lists_from_state_fidelity_list(state_fidelity_list: list, dim: int):
    # calculate the average gate fidelity with arithmetic mean
    fidelity_arith = 1 / ( dim + 1 ) * ( sum(state_fidelity_list) )

    # calculate the average gate fidelity with modified geometric mean
    fidelity_prod = np.prod(state_fidelity_list[:-1])
    fidelity_geom = 1 / ( dim + 1 ) + (1 - 1 / ( dim + 1 )) * state_fidelity_list[-1] * fidelity_prod

    # calculate the average gate fidelity with a combined measure of two
    lambda_mix = 1 - (1 - fidelity_prod) / (1 - fidelity_prod * state_fidelity_list[-1])
    # lambda_mix = fidelity_prod * (1 - state_fidelity_list[-1]) / (1 - fidelity_prod * state_fidelity_list[-1])
    fidelity_lambda = lambda_mix * fidelity_geom + (1 - lambda_mix) * fidelity_arith

    return fidelity_arith, fidelity_geom, fidelity_lambda

In [ ]:
state_fidelity_list = compute_state_fidelity_list(gate1, gate2, dim, input_dm_list)
print(state_fidelity_list)
fidelity_arith, fidelity_geom, fidelity_lambda = compute_gate_fidelity_lists_from_state_fidelity_list(state_fidelity_list, dim)
print(fidelity_arith, fidelity_geom, fidelity_lambda)

[0.10609271450666462, 0.10609271450666462, 0.9958081267034815]
0.4026645185722702 0.34080565450249145 0.4026615668432955


#### Step 2 : Use minimize to find a set of optimal angles. Using a callback function, save the fidelity for each iteration. Plot the fidelity versus iteration number. Try with various solvers within minimize ( i.e., Nelder-Mead, L-BFGS-B, Powell )

In [69]:
U_target = sigmax()

def gate_fidelity_arith(alpha, beta, gamma):
    U_optim = get_zxz_gate(alpha, beta, gamma)
    
    state_fidelity_list = []
    for rho in input_dm_list:
        rho_optim = U_optim * rho * U_optim.dag()
        rho_target = U_target * rho * U_target.dag()
        state_fidelity_list.append(float(fidelity(rho_optim, rho_target)))

    fidelity_arith = compute_gate_fidelity_arith_from_state_fidelity_list(state_fidelity_list, dim)

    return fidelity_arith

# Parameter ranges (make sure beta range includes pi)
bounds = [(-np.pi, np.pi), (0.0, np.pi), (-np.pi, np.pi)]

# callback function to monitor optimization progress
fid_history = []
def callback(xk):
    fid = gate_fidelity_arith(xk[0], xk[1], xk[2])
    fid_history.append(fid)
    print(f"Current params: alpha={xk[0]:.4f}, beta={xk[1]:.4f}, gamma={xk[2]:.4f} => Fidelity={fid:.6f}")

In [ ]:
res_BFGS = minimize(
    lambda x: 1 - gate_fidelity_arith(x[0], x[1], x[2]),
    x0 = [0, 2.5, 0],
    method = "L-BFGS-B",
    bounds = bounds,
    callback=callback,
    options={"maxiter": 500, "disp": True},
)

Current params: alpha=0.0000, beta=2.5507, gamma=0.1756 => Fidelity=0.969826
Current params: alpha=0.2605, beta=2.8299, gamma=0.3255 => Fidelity=0.991413
Current params: alpha=0.2765, beta=2.9031, gamma=0.5480 => Fidelity=0.991858
Current params: alpha=0.5858, beta=3.1416, gamma=0.9792 => Fidelity=0.993573
Current params: alpha=0.5858, beta=3.1416, gamma=0.9792 => Fidelity=0.993573


C:\Windows\Temp\ipykernel_6272\958394004.py:1: DeprecationWarning: scipy.optimize: The `disp` and `iprint` options of the L-BFGS-B solver are deprecated and will be removed in SciPy 1.18.0.
  res = minimize(
d:\anaconda3\envs\my_qiskit\Lib\site-packages\qutip\core\data\expm.py:139: LinAlgWarning: Matrix is singular. The result might be inaccurate or the array might not have a square root.
  return Dense(scipy.linalg.sqrtm(matrix.as_ndarray()))


In [ ]:
a_opt, b_opt, g_opt = res_BFGS.x
F_opt = gate_fidelity_arith(a_opt, b_opt, g_opt)

print("\n===== Optimization Result by L-BFGS-B =====")
print("success:", res_BFGS.success)
print("message:", res_BFGS.message)
print(f"optimal angles (rad): alpha={a_opt:.12f}, beta={b_opt:.12f}, gamma={g_opt:.12f}")
print(f"max gate_fidelity (arith): {F_opt:.12f}")
print(f"min objective (1-F): {res_BFGS.fun:.3e}")

# # If you want to see the fidelity per iteration:
# if len(fid_history) > 0:
#     print("\nIterations recorded by callback:", len(fid_history))
#     print("first few fidelities:", fid_history[:5])
#     print("last fidelity:", fid_history[-1])


===== Optimization Result =====
success: True
message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
optimal angles (rad): alpha=0.585774869566, beta=3.141592653590, gamma=0.979161475027
max gate_fidelity (arith): 0.993572721376
min objective (1-F): 6.427e-03


In [71]:
res_Powell = minimize(
    lambda x: 1 - gate_fidelity_arith(x[0], x[1], x[2]),
    x0 = [0.1, 2.5, 1],
    method = "Powell",
    bounds = bounds,
    callback=callback,
    options={"maxiter": 500, "disp": True},
)

a_opt, b_opt, g_opt = res_Powell.x
F_opt = gate_fidelity_arith(a_opt, b_opt, g_opt)

print("\n===== Optimization Result by Powell =====")
print("success:", res_Powell.success)
print("message:", res_Powell.message)
print(f"optimal angles (rad): alpha={a_opt:.12f}, beta={b_opt:.12f}, gamma={g_opt:.12f}")
print(f"max gate_fidelity (arith): {F_opt:.12f}")
print(f"min objective (1-F): {res_Powell.fun:.3e}")

Current params: alpha=0.8952, beta=3.1415, gamma=0.8952 => Fidelity=1.000000
Current params: alpha=0.8956, beta=3.1414, gamma=0.8952 => Fidelity=1.000000
Current params: alpha=0.8955, beta=3.1415, gamma=0.8960 => Fidelity=1.000000
Current params: alpha=0.8960, beta=3.1415, gamma=0.8961 => Fidelity=1.000000
Current params: alpha=0.8960, beta=3.1414, gamma=0.8961 => Fidelity=1.000000
Optimization terminated successfully.
         Current function value: -0.000000
         Iterations: 5
         Function evaluations: 308

===== Optimization Result by Powell =====
success: True
message: Optimization terminated successfully.
optimal angles (rad): alpha=0.896006202665, beta=3.141419648519, gamma=0.896112902467
max gate_fidelity (arith): 1.000000021538
min objective (1-F): -2.154e-08


In [73]:
res_Nelder = minimize(
    lambda x: 1 - gate_fidelity_arith(x[0], x[1], x[2]),
    x0 = [0.1, 2.5, -0.1],
    method = "Nelder-Mead",
    bounds = bounds,
    callback=callback,
    options={"maxiter": 500, "disp": True},
)

a_opt, b_opt, g_opt = res_Nelder.x
F_opt = gate_fidelity_arith(a_opt, b_opt, g_opt)

print("\n===== Optimization Result by Nelder-Mead =====")
print("success:", res_Nelder.success)
print("message:", res_Nelder.message)
print(f"optimal angles (rad): alpha={a_opt:.12f}, beta={b_opt:.12f}, gamma={g_opt:.12f}")
print(f"max gate_fidelity (arith): {F_opt:.12f}")
print(f"min objective (1-F): {res_Nelder.fun:.3e}")

Current params: alpha=0.1000, beta=2.6250, gamma=-0.1000 => Fidelity=0.976328
Current params: alpha=0.0950, beta=2.7083, gamma=-0.0933 => Fidelity=0.983010
Current params: alpha=0.0900, beta=2.9167, gamma=-0.0967 => Fidelity=0.994356
Current params: alpha=0.0950, beta=3.0833, gamma=-0.0833 => Fidelity=0.998394
Current params: alpha=0.0800, beta=3.1416, gamma=-0.0733 => Fidelity=0.999021
Current params: alpha=0.0800, beta=3.1416, gamma=-0.0733 => Fidelity=0.999021
Current params: alpha=0.0767, beta=3.1416, gamma=-0.0389 => Fidelity=0.999444
Current params: alpha=0.0483, beta=3.1416, gamma=-0.0211 => Fidelity=0.999799
Current params: alpha=0.0417, beta=3.1416, gamma=0.0178 => Fidelity=0.999976
Current params: alpha=0.0311, beta=3.1416, gamma=0.0452 => Fidelity=0.999992
Current params: alpha=0.0311, beta=3.1416, gamma=0.0452 => Fidelity=0.999992
Current params: alpha=0.0311, beta=3.1416, gamma=0.0452 => Fidelity=0.999992
Current params: alpha=0.0311, beta=3.1416, gamma=0.0452 => Fidelity=

d:\anaconda3\envs\my_qiskit\Lib\site-packages\qutip\core\data\expm.py:139: LinAlgWarning: Matrix is singular. The result might be inaccurate or the array might not have a square root.
  return Dense(scipy.linalg.sqrtm(matrix.as_ndarray()))
